In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tempfile

In [4]:
tempfile.gettempdir()

'C:\\Users\\ACERM5~1\\AppData\\Local\\Temp'

### Data Camp Article

[link](https://www.datacamp.com/community/tutorials/cnn-tensorflow-python#cnn)

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [6]:
print('Train Set Shape:',x_train.shape,'Test Set Shape:',x_test.shape)
print('Train Targets Set Shape:',y_train.shape,'Test Targets Set Shape:',y_test.shape)

Train Set Shape: (60000, 28, 28) Test Set Shape: (10000, 28, 28)
Train Targets Set Shape: (60000,) Test Targets Set Shape: (10000,)


In [7]:
x_train,x_test = x_train/255, x_test/255                 ### Rescaling betwen 0 and 1 (Normalizing)

In [8]:
x_train= x_train.reshape(-1,28,28,1)
x_test= x_test.reshape(-1,28,28,1)

In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot= OneHotEncoder()
y_train= one_hot.fit_transform(y_train.reshape(-1,1))
y_test= one_hot.fit_transform(y_test.reshape(-1,1))

C:\Users\ACER M5\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\ACER M5\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "catego

In [10]:
print(y_train.shape,y_test.shape)

(60000, 10) (10000, 10)


In [11]:
y_train=y_train.toarray()
y_test=y_test.toarray()

In [14]:
x_test= x_test[:200]
y_test=y_test[:200]

### The Deep Neural Network

You'll use three convolutional layers:

...> The first layer will have 32-3 x 3 filters,

...> The second layer will have 64-3 x 3 filters and

...> The third layer will have 128-3 x 3 filters.

...>In addition, there are three max-pooling layers each of size 2 x 2.

In [15]:
total_epochs = 40 
learning_rate = 0.001 
batch_size = 256           ### batch size must be a power of 2

In [16]:
n_inputs= 28
n_classes= 10

In [17]:
x= tf.placeholder('float')
y= tf.placeholder('float')

In your network architecture model, you will have multiple convolution and max-pooling layers. In such cases, it's always a better idea to define convolution and max-pooling functions, so that you can call them as many times you want to use them in your network

In [18]:
def conv2d(x,W,b,strides=1):
    
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

In [19]:
## shape(filter,filter,channels,no.of convolutional filters)
## Next consective layer will have channels equal to previous convolutional Network
## You can either initiaze it here or latter in the Session

weights = {
    'wc1': tf.get_variable('weight_1', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('weight_2',shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('weight_3', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('dense_1',shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('output',shape=(128,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable( 'biase_1',shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('biase_2' ,shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('biase_3',shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable( 'dense_biase',shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('output_biase' ,shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
def conv_net(x, weights, biases):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
   # fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1= tf.reshape(conv3,[-1,4*4*128])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out


In [21]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

saver= tf.train.Saver()

In [23]:
total_epochs= 10

with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        summary_writer = tf.summary.FileWriter('Output', sess.graph)
        for epoch in range(total_epochs):
            
            epoch_loss = 0
            for _ in range(int(len(x_train)/ batch_size)):
                epoch_x = x_train[_*batch_size:min((_+1)*batch_size,len(x_train))]
                epoch_y = y_train[_*batch_size:min((_+1)*batch_size,len(y_train))]  
                _,c = sess.run([optimizer,cost], feed_dict={x:epoch_x,y:epoch_y})
                epoch_loss += c
                
            print('Epoch' , epoch ,'completed out of' , total_epochs, 'Loss' , epoch_loss)
            
            
        correct= tf.equal(tf.arg_max(pred,1),tf.arg_max(y,1))
        accuracy= tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:', accuracy.eval({x:x_test,y:y_test}))
        
        save_path = saver.save(sess, "E:/tmp/model.ckpt")
        print("Model saved in path: %s" % save_path)
        
        summary_writer.close()
            

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 10 Loss 161.24393266439438
Epoch 1 completed out of 10 Loss 88.82167971134186
Epoch 2 completed out of 10 Loss 74.7361772954464
Epoch 3 completed out of 10 Loss 67.6044419258833
Epoch 4 completed out of 10 Loss 62.48726274073124
Epoch 5 completed out of 10 Loss 58.343036606907845
Epoch 6 completed out of 10 Loss 54.753327772021294
Epoch 7 completed out of 10 Loss 51.67805731296539
Epoch 8 completed out of 10 Loss 49.282560095191
Epoch 9 completed out of 10 Loss 46.511997640132904
Instructions for updating:
Use `tf.math.argmax` instead
Accuracy: 0.905
Model saved in path: E:/tmp/model.ckpt
